In [ ]:
%config Completer.use_jedi = False

<h1><center>Logistic Regression Baseline based on Stats only</center></h1>
                           
                           
<center><img src = "https://storage.googleapis.com/kaggle-competitions/kaggle/31779/logos/header.png" width = "1000" height = "400"/></center>    


<h3 style='background:orange; color:black'><center>Consider upvoting this notebook if you found it helpful.</center></h3>

In [ ]:
import os
import shutil

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
files_train = os.listdir('/kaggle/input/feedback-prize-2021/train')
files_test = os.listdir('/kaggle/input/feedback-prize-2021/test')

In [ ]:
df = pd.read_csv('/kaggle/input/feedback-prize-2021/train.csv')

In [ ]:
label_ordered_list = df['discourse_type'].value_counts().index.tolist()[::-1]

In [ ]:
df['char_len'] = (df['discourse_end'] - df['discourse_start']).astype(int)
df['word_len'] = df['predictionstring'].str.split().apply(len)

In [ ]:
label_ratio = []
for i, txt in tqdm(df.groupby('id')): 
    
    txt_id = txt['id'].values[0]
    txt_file = f"/kaggle/input/feedback-prize-2021/train/{txt_id}.txt"
    
    with open(txt_file, 'r') as file:
        txt_data = file.read()
        
    len_lbls = txt['char_len'].sum()
    len_txt = len(txt_data)
    ratio = len_lbls/len_txt

    label_ratio.append(pd.DataFrame({'id': [txt_id], 'ratio':[ratio]}))
    
label_ratio = pd.concat(label_ratio).reset_index(drop = True)

In [ ]:
files_to_keep = label_ratio['id'][label_ratio['ratio'] > 0.80]
df_clean = df.set_index('id').loc[files_to_keep].reset_index().copy()

In [ ]:
df_clean = df_clean.set_index('id')
for idx, group in tqdm(df_clean.groupby(df_clean.index)):
    txt_file_path = f'/kaggle/input/feedback-prize-2021/train/{idx}.txt'
    with open(txt_file_path, 'r') as file:
        text_data = file.read()
    txt_len = len(text_data.split())
    df_clean.loc[idx, 'txt_len'] = int(txt_len)
df_clean = df_clean.reset_index()

In [ ]:
df_clean['predictionarray'] = df_clean.progress_apply(lambda x: np.array(x['predictionstring'].split(), dtype = int) / x['txt_len'], axis = 1)

In [ ]:
df_clean = df_clean[['predictionarray', 'discourse_type']].explode('predictionarray')
df_clean['discourse_type'] = df_clean['discourse_type'].astype('category')

In [ ]:
X = df_clean[['predictionarray']]
y = df_clean['discourse_type']

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', verbose = 1, n_jobs = 4)

In [ ]:
model.fit(X, y)

In [ ]:
import matplotlib.pyplot as plt

X_test = np.expand_dims(np.linspace(0, 1, num = 1001), axis = 1)
y_test = model.predict_proba(X_test).argmax(axis = -1)

labels = y.cat.categories
plt.plot(X_test, y_test)
plt.yticks(range(len(labels)), labels);

In [ ]:
sub = []
for txt_file in files_test:
    
    txt_file_path = f'/kaggle/input/feedback-prize-2021/test/{txt_file}'
    with open(txt_file_path, 'r') as file:
        text_data = file.read()
    txt_len = len(text_data.split())
    
    X_test = np.expand_dims((np.arange(txt_len) + 1) / txt_len, axis = 1)
    y_test = model.predict_proba(X_test).argmax(axis = -1)

    predictions = {label: ' '.join(np.where(y_test == i)[0].astype(str)) for i, label in enumerate(labels)}

    sub.append(pd.DataFrame({
        'id': txt_file.split('.')[0],
        'class': predictions.keys(),
        'predictionstring': predictions.values()
    }))
    
    
submission = pd.concat(sub).reset_index(drop = True)
submission = submission[submission['predictionstring'] != '']

In [ ]:
submission.to_csv('submission.csv', index = False)
submission

<h3 style='background:orange; color:black'><center>WORK IN PROGRESS!! Come back for more later...</center></h3>